In [328]:
from tools.ground_removal import *
from tools.voxelization import *
from tools.pedestrians import *
from sklearn.cluster import DBSCAN
import numpy as np
from mayavi import mlab

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [621]:
pts_, intens = get_synchronized_frames(0,10)
#mask = (pts_[:,2] < 1) & (pts_[:,2] > -1)
#pts = pts_[mask]
pts = pts_

In [581]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()



In [499]:

mask = (pts[:,2] > -0.7) & (pts[:,2] < 1)
cropped = pts[mask]

In [497]:
clustering = DBSCAN(eps=0.3, min_samples=10 * 10, ).fit(cropped[:, :3])

In [501]:
mlab.points3d(cropped[:,0], cropped[:,1], cropped[:,2],clustering.labels_, mode='point')
mlab.show()

In [634]:
pts_, intens = get_synchronized_frames(2,10)
cropped_pts, dynamic_mask, clustering, dynamic_clusters, differences, centroids_final = find_pedestrians(pts_[:,:3], pts_[:,3].reshape(-1,1))

formed 60 clusters
skipping cluster 0 because of it's size w 17.388552952926617, l 3.27106698748541, h 1.9979182792446202
skipping cluster 1 at time 0 because of it's size w 8.83554864233319, l 2.234773556500903, h 1.5161690367793579
skipping cluster 2 at time 0 because of it's size w 1.786158517855947, l 4.15531386458175, h 1.6470744582625976
cluster 3, time 5 skipping because no points; cluster 3, time 6 skipping because no points; cluster 3, time 7 skipping because no points; cluster 3, time 8 skipping because no points; cluster 3, time 9 skipping because no points; 

skipping cluster 4 at time 0 because of it's size w 0.6724336845215255, l 2.434623999234912, h 1.331695346211621
cluster 5, time 5 skipping because no points; cluster 5, time 6 skipping because no points; cluster 5, time 7 skipping because no points; cluster 5, time 8 skipping because no points; cluster 5, time 9 skipping because no points; 

skipping cluster 6 because of it's size w 1.622983503489384, l 2.118072176118

In [635]:
print(dynamic_clusters)
pts = cropped_pts

[31, 35, 42, 45]


In [636]:
mlab.points3d(cropped_pts[:,0][dynamic_mask], cropped_pts[:,1][dynamic_mask],
              cropped_pts[:,2][dynamic_mask], cropped_pts[:,3][dynamic_mask], mode='point')
mlab.points3d(cropped_pts[:,0][~dynamic_mask], cropped_pts[:,1][~dynamic_mask], 
              cropped_pts[:,2][~dynamic_mask], mode='point')
mlab.show()

In [625]:
cluster = 48 #dynamic_clusters[4]
print(f"average speed {np.linalg.norm(differences[cluster][:2])}")
single_mask = clustering.labels_ == cluster
time_mask = pts[:,3][single_mask] == 0 # at time 0

mlab.points3d(pts[:,0][single_mask][time_mask], pts[:,1][single_mask][time_mask],
              pts[:,2][single_mask][time_mask], pts[:,3][single_mask][time_mask],
              mode='point')

mlab.points3d(pts[:,0][~single_mask], pts[:,1][~single_mask], pts[:,2][~single_mask],
              mode='point')

#mlab.points3d(pts_[:,0], pts_[:,1], pts_[:,2], mode='point')
mlab.show()

average speed 0.08243254823794736


In [599]:
bb_x_min =  pts[:,0][single_mask][time_mask].min()
bb_x_max =  pts[:,0][single_mask][time_mask].max()

bb_y_min =  pts[:,1][single_mask][time_mask].min()
bb_y_max =  pts[:,1][single_mask][time_mask].max()

bb_z_min =  pts[:,2][single_mask][time_mask].min()
bb_z_max =  pts[:,2][single_mask][time_mask].max()

In [512]:

# along x axis
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_min, bb_y_min], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_max, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_min, bb_y_min], [bb_z_max, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_max, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))

# along y axis
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))

# along z axis
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_min], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_max, bb_y_max], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_min], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_max, bb_y_max], [bb_z_min, bb_z_max], color=(1,0.7,0.3))


mlab.points3d(pts[:,0][single_mask][time_mask], pts[:,1][single_mask][time_mask],
              pts[:,2][single_mask][time_mask], pts[:,3][single_mask][time_mask],
              mode='point')
mlab.show()

In [567]:
bb_mask = (pts_[:,0] < bb_x_max) & (pts_[:,0] > bb_x_min) \
    & (pts_[:,1] < bb_y_max) & (pts_[:,1] > bb_y_min)

floor = pts_[:,2][bb_mask].min()
bb_mask = bb_mask & (pts_[:,2] < (floor + 0.1))

In [568]:
mlab.points3d(pts_[:,0][bb_mask], pts_[:,1][bb_mask],
              pts_[:,2][bb_mask], pts_[:,3][bb_mask],
              mode='point')
mlab.points3d(pts_[:,0][~bb_mask], pts_[:,1][~bb_mask],
              pts_[:,2][~bb_mask],
              mode='point')
mlab.show()


In [552]:
np.sum(pts_[:,3][bb_mask] == 3)

0

In [575]:
v,f = np.unique(pts_[:,3][bb_mask], return_counts=True)
v[f>1]


array([0., 2., 4., 5., 6., 8., 9.])

In [402]:
pts_[:,3][bb_mask]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
       9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
       9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.])

In [569]:
for i,j in zip(v,f):
    print(i)

0.0
2.0
4.0
5.0
6.0
8.0
9.0


In [615]:
bb_mask = (pts_[:,0] < bb_x_max) & (pts_[:,0] > bb_x_min) \
    & (pts_[:,1] < bb_y_max) & (pts_[:,1] > bb_y_min) \
    & (pts_[:,2] < bb_z_max) & (pts_[:,2] > bb_z_min)
np.sum((pts_[:,3] == 5)[bb_mask])

4